In [1]:
# # Import packages
# import pandas as pd
# import numpy as np
# import os
# import matplotlib.pyplot as plt
# import seaborn as sns
# import xgboost as xgb
# 
# # Set options
# pd.options.display.max_rows = 999
# pd.options.display.max_columns = 999
# 
# train_x_raw = pd.read_csv("../01-Data/X_train.csv", low_memory = True, index_col=0)
# train_y_raw = pd.read_csv("../01-Data/y_train.csv", low_memory = True, index_col=0)
# test_x_raw = pd.read_csv("../01-Data/X_test.csv", low_memory=True, index_col=0)
# 
# df_train = pd.DataFrame(train_x_raw)
# df_test = pd.DataFrame(test_x_raw)
# df_y = pd.DataFrame(train_y_raw)

from EDA_script import *

# Model Run
##  Random Forest with RandomizedSearchCV

In [2]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import scipy.stats as stats
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

label_mapping = {-1: 0, 1: 1, 2: 2, 3: 3, 4: 4}
df_y = df_y.replace(label_mapping)

X_train, X_val, y_train, y_val = train_test_split(df_train, df_y.squeeze(), test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)

# Define the distribution of parameters
param_distributions = {
    'n_estimators': stats.randint(100, 500),
    'max_depth': stats.randint(4, 10),
    'min_samples_split': stats.randint(2, 11),
    'min_samples_leaf': stats.randint(1, 11),
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

# RandomizedSearchCV / change n_iter, cv
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=100, cv=3, scoring='neg_log_loss', random_state=42, verbose=2)
random_search.fit(X_train, y_train)

# Best parameters found
print("Best parameters:", random_search.best_params_)

# Best score achieved
print("Best score:", random_search.best_score_)

best_params = random_search.best_params_
# RandomForestClassifier with the best parameters
best_model = RandomForestClassifier(**best_params, random_state=42)

# Fit the model
best_model.fit(df_train, df_y.squeeze())  

y_test_probs = best_model.predict_proba(X_val)  
test_log_loss = log_loss(y_val, y_test_probs)  
print(f"Test Multiclass Logarithmic Loss: {test_log_loss}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END bootstrap=True, criterion=entropy, max_depth=8, max_features=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002201FB85240>, min_samples_leaf=8, min_samples_split=6, n_estimators=202; total time=   0.0s
[CV] END bootstrap=True, criterion=entropy, max_depth=8, max_features=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002201FB85240>, min_samples_leaf=8, min_samples_split=6, n_estimators=202; total time=   0.0s
[CV] END bootstrap=True, criterion=entropy, max_depth=8, max_features=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002201FB85240>, min_samples_leaf=8, min_samples_split=6, n_estimators=202; total time=   0.0s
[CV] END bootstrap=False, criterion=gini, max_depth=6, max_features=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002201FB85240>, min_samples_leaf=8, min_samples_split=6, n_estimators=199; total time=   0.0s
[

C:\Users\amyoo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
12 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\amyoo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\amyoo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\amyoo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py"

Best parameters: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 463}
Best score: -1.0233345689493245
Test Multiclass Logarithmic Loss: 1.006397410185323
